`python -m venv venv`

`.\venv\Scripts\activate`

In [47]:
pip install pandas numpy matplotlib seaborn scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [48]:
#import necessary modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.preprocessing import OneHotEncoder

In [49]:
df = pd.read_excel("../files/titanic3.xls")
df.head(10)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
5,1,1,"Anderson, Mr. Harry",male,48.0000,0,0,19952,26.5500,E12,S,3,NaN,"New York, NY"
6,1,1,"Andrews, Miss. Kornelia Theodosia",female,63.0000,1,0,13502,77.9583,D7,S,10,NaN,"Hudson, NY"
7,1,0,"Andrews, Mr. Thomas Jr",male,39.0000,0,0,112050,0.0000,A36,S,NaN,NaN,"Belfast, NI"
8,1,1,"Appleton, Mrs. Edward Dale (Charlotte Lamson)",female,53.0000,2,0,11769,51.4792,C101,S,D,NaN,"Bayside, Queens, NY"
9,1,0,"Artagaveytia, Mr. Ramon",male,71.0000,0,0,PC 17609,49.5042,NaN,C,NaN,22.0,"Montevideo, Uruguay"


In [62]:
# check dimensions
df.info()
df.isnull().any()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype             
---  ------      --------------  -----             
 0   pclass      1309 non-null   int64             
 1   survived    1309 non-null   int64             
 2   name        1309 non-null   object            
 3   age         1309 non-null   float64           
 4   sibsp       1309 non-null   int64             
 5   parch       1309 non-null   int64             
 6   ticket      1309 non-null   object            
 7   fare        1309 non-null   float64           
 8   cabin       1309 non-null   object            
 9   embarked    1309 non-null   object            
 10  boat        486 non-null    object            
 11  body        121 non-null    float64           
 12  home.dest   1309 non-null   object            
 13  title       1309 non-null   object            
 14  sex_female  1 non-null      Sparse[float64, 0]
 15  sex_

C:\Users\School\AppData\Local\Temp\ipykernel_18416\3503718380.py:2: FutureWarning: Allowing arbitrary scalar fill_value in SparseDtype is deprecated. In a future version, the fill_value must be a valid value for the SparseDtype.subtype.
  df.info()


pclass        False
survived      False
name          False
age           False
sibsp         False
parch         False
ticket        False
fare          False
cabin         False
embarked      False
boat           True
body           True
home.dest     False
title         False
sex_female    False
sex_male      False
dtype: bool

In [51]:
#find the title for each person
def find_title(name):
    strings = name.split()
    for string in strings:
        if string.endswith('.'):
            return string
        
    return None
df['title'] = df['name'].apply(find_title)

df.head()


,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,title
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO",Miss.
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON",Master.
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON",Miss.
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON",Mr.
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON",Mrs.


In [52]:
title_avg_age = df.groupby('title')['age'].mean()
# Fill in missing ages based on title with the average age
df = df.merge(title_avg_age, on='title', suffixes=('', '_avg'))
df['age'].fillna(df['age_avg'], inplace=True)
df.drop(columns=['age_avg'], inplace=True)

print(df)
    

      pclass  survived                               name     sex   age  \
0          1         1      Allen, Miss. Elisabeth Walton  female  29.0   
1          1         0       Allison, Miss. Helen Loraine  female   2.0   
2          1         1  Andrews, Miss. Kornelia Theodosia  female  63.0   
3          1         1       Barber, Miss. Ellen "Nellie"  female  26.0   
4          1         1              Bazzani, Miss. Albina  female  32.0   
...      ...       ...                                ...     ...   ...   
1304       2         0             Lahtinen, Rev. William    male  30.0   
1305       2         0              Montvila, Rev. Juozas    male  27.0   
1306       2         0      Peruschitz, Rev. Joseph Maria    male  41.0   
1307       2         1          Reynaldo, Ms. Encarnacion  female  28.0   
1308       3         0            O'Donoghue, Ms. Bridget  female  28.0   

      sibsp  parch  ticket      fare    cabin embarked boat  body  \
0         0      0   24160  21

In [53]:
#fill in average fare with total average
avg_fare = df['fare'].mean()

df['fare'].fillna(avg_fare, inplace=True)

In [54]:

df['cabin'] = df['cabin'].fillna('X')

df['cabin'] = df['cabin'].apply(lambda x: str(x)[0])

df['embarked'] = df['embarked'].fillna('X')

df['home.dest'] = df['home.dest'].fillna('X')

home_value_counts = df['home.dest'].value_counts()
print(home_value_counts)



embarking_value_counts = df['embarked'].value_counts()
print(embarking_value_counts)

# df['embarked'] = df[embarking_value_counts[0]]

# embarking_value_counts = df['embarked'].value_counts()

# print(embarking_value_counts)




home.dest
X                           564
New York, NY                 64
London                       14
Montreal, PQ                 10
Paris, France                 9
                           ... 
Clevedon, England             1
Auburn, NY                    1
Seattle, WA / Toledo, OH      1
Walthamstow, England          1
Spain                         1
Name: count, Length: 370, dtype: int64
embarked
S    914
C    270
Q    123
X      2
Name: count, dtype: int64


In [55]:
# check dimensions
df.info()
df.isnull().any()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     1309 non-null   int64  
 1   survived   1309 non-null   int64  
 2   name       1309 non-null   object 
 3   sex        1309 non-null   object 
 4   age        1309 non-null   float64
 5   sibsp      1309 non-null   int64  
 6   parch      1309 non-null   int64  
 7   ticket     1309 non-null   object 
 8   fare       1309 non-null   float64
 9   cabin      1309 non-null   object 
 10  embarked   1309 non-null   object 
 11  boat       486 non-null    object 
 12  body       121 non-null    float64
 13  home.dest  1309 non-null   object 
 14  title      1309 non-null   object 
dtypes: float64(3), int64(4), object(8)
memory usage: 153.5+ KB


pclass       False
survived     False
name         False
sex          False
age          False
sibsp        False
parch        False
ticket       False
fare         False
cabin        False
embarked     False
boat          True
body          True
home.dest    False
title        False
dtype: bool

In [56]:
df.head()
print(df['cabin'].unique())
print(df['embarked'].unique())

['B' 'C' 'D' 'X' 'E' 'A' 'F' 'G' 'T']
['S' 'C' 'X' 'Q']


In [57]:
df['sex'] = df['sex'].astype('category')
enc = OneHotEncoder()

df_encoded =  enc.fit_transform(df[['sex']])
sex_encoded_df = pd.DataFrame.sparse.from_spmatrix(df_encoded, columns=enc.get_feature_names_out(['sex']))

df = pd.concat([df, sex_encoded_df], axis=1)

In [58]:
df = df.drop('sex', axis=1)

In [59]:
print(df.head())

   pclass  survived                               name   age  sibsp  parch  \
0       1         1      Allen, Miss. Elisabeth Walton  29.0      0      0   
1       1         0       Allison, Miss. Helen Loraine   2.0      1      2   
2       1         1  Andrews, Miss. Kornelia Theodosia  63.0      1      0   
3       1         1       Barber, Miss. Ellen "Nellie"  26.0      0      0   
4       1         1              Bazzani, Miss. Albina  32.0      0      0   

   ticket      fare cabin embarked boat  body  \
0   24160  211.3375     B        S    2   NaN   
1  113781  151.5500     C        S  NaN   NaN   
2   13502   77.9583     D        S   10   NaN   
3   19877   78.8500     X        S    6   NaN   
4   11813   76.2917     D        C    8   NaN   

                         home.dest  title  sex_female  sex_male  
0                     St Louis, MO  Miss.         1.0       0.0  
1  Montreal, PQ / Chesterville, ON  Miss.         1.0       0.0  
2                       Hudson, NY  Mi